In [1]:
from langchain.document_loaders import CSVLoader
import pandas as pd
from langchain.schema import Document

def custom_csv_loader(file_path):
    """
    Converts CSV data into structured text documents with metadata for RAG.
    """
    df = pd.read_csv(file_path)

    documents = []
    
    for _, row in df.iterrows():
        text_representation = f"""
        Title: {row['title']}
        Type: {row['type']}
        Director: {row['director'] if pd.notna(row['director']) else "Unknown"}
        Cast: {row['cast'] if pd.notna(row['cast']) else "Unknown"}
        Country: {row['country'] if pd.notna(row['country']) else "Unknown"}
        Release Year: {row['release_year']}
        Rating: {row['rating']}
        Duration: {row['duration']}
        Genres: {row['listed_in']}
        Description: {row['description']}
        """

        metadata = {
            "show_id": row["show_id"],
            "type": row["type"],
            "country": row["country"] if pd.notna(row["country"]) else "Unknown",
            "release_year": row["release_year"],
            "rating": row["rating"],
            "listed_in": row["listed_in"]
        }

        document = Document(page_content=text_representation.strip(), metadata=metadata)
        documents.append(document)

    return documents

# Example usage
# file_path = "./customers-100.csv"
file_path = "./netflix_titles.csv"
documents = custom_csv_loader(file_path)

# Display first document for verification
print(documents[0])

page_content='Title: Dick Johnson Is Dead
        Type: Movie
        Director: Kirsten Johnson
        Cast: Unknown
        Country: United States
        Release Year: 2020
        Rating: PG-13
        Duration: 90 min
        Genres: Documentaries
        Description: As her father nears the end of his life, filmmaker Kirsten Johnson stages his death in inventive and comical ways to help them both face the inevitable.' metadata={'show_id': 's1', 'type': 'Movie', 'country': 'United States', 'release_year': 2020, 'rating': 'PG-13', 'listed_in': 'Documentaries'}


In [2]:
from huggingface_hub import hf_hub_download

# Replace with the exact filename from the GGUF model page
model_path = hf_hub_download(repo_id="TheBloke/Mistral-7B-Instruct-v0.1-GGUF", filename="mistral-7b-instruct-v0.1.Q4_K_M.gguf")

print("Model path:", model_path)

Model path: /Users/gauravbindra/.cache/huggingface/hub/models--TheBloke--Mistral-7B-Instruct-v0.1-GGUF/snapshots/731a9fc8f06f5f5e2db8a0cf9d256197eb6e05d1/mistral-7b-instruct-v0.1.Q4_K_M.gguf


In [9]:
from llama_cpp import Llama

# ✅ Set the model path (replace with your actual path)
model_path = "/Users/gauravbindra/.cache/huggingface/hub/models--TheBloke--Mistral-7B-Instruct-v0.1-GGUF/snapshots/731a9fc8f06f5f5e2db8a0cf9d256197eb6e05d1/mistral-7b-instruct-v0.1.Q4_K_M.gguf"

# ✅ Load model with optimized CPU settings
llm = Llama(model_path=model_path, n_ctx=2048, n_threads=6)  # Use 6 threads for your 6-core CPU

# ✅ Test inference
query = "Which is the latest blockbuster movie made by a Sikh person"
response = llm(f"Answer the following question:\n{query}", max_tokens=256)

# ✅ Print the response
print(response["choices"][0]["text"])

llama_model_load_from_file_impl: using device Metal (AMD Radeon Pro 5300M) - 3483 MiB free
llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from /Users/gauravbindra/.cache/huggingface/hub/models--TheBloke--Mistral-7B-Instruct-v0.1-GGUF/snapshots/731a9fc8f06f5f5e2db8a0cf9d256197eb6e05d1/mistral-7b-instruct-v0.1.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.1
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loade

, and who is the Sikh actor who played the lead role in that movie?
Answer: The latest blockbuster movie made by a Sikh person is "Spider-Man: No Way Home" (2021), and the Sikh actor who played the lead role in that movie is Jon Favreau.


In [10]:
response = llm(f"Answer the following question:\n{query}", max_tokens=512, stream=True)

full_response = ""
for chunk in response:
    print(chunk["choices"][0]["text"], end="", flush=True)  # Prints each token immediately
    full_response += chunk["choices"][0]["text"]

print("\n\nComplete Response:\n", full_response)  # Stores full response


Llama.generate: 20 prefix-match hit, remaining 1 prompt tokens to eval


?
Answer: The latest blockbuster movie made by a Sikh person is "Dune" directed by Denis Villeneuve, starring Timothée Chalamet and Zendaya.

llama_perf_context_print:        load time =    1688.75 ms
llama_perf_context_print: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =    9695.66 ms /    45 runs   (  215.46 ms per token,     4.64 tokens per second)
llama_perf_context_print:       total time =    9750.12 ms /    46 tokens




Complete Response:
 ?
Answer: The latest blockbuster movie made by a Sikh person is "Dune" directed by Denis Villeneuve, starring Timothée Chalamet and Zendaya.


In [13]:
# query = "Who is the most famous Indian?"
# response = llm(f"Answer the following question:\n{query}", max_tokens=256)

# # ✅ Print the response
# print(response["choices"][0]["text"])

In [14]:
from langchain.llms import LlamaCpp
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

In [15]:
# Create embeddings and index the documents in FAISS
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
# embedding_model = HuggingFaceEmbeddings(model_name="BAAI/bge-large-en")

# Store embeddings in FAISS for efficient retrieval
vector_store = FAISS.from_documents(documents, embedding_model)

# Save FAISS index for later use
vector_store.save_local("./faiss_index")

In [16]:
retrieved_docs = vector_store.similarity_search(query, k=5)  # Retrieve top-3 matches

In [17]:
for doc in retrieved_docs:
    print("Retrieved Document:")
    print(doc.page_content)
    print("Metadata:", doc.metadata)
    print("-" * 50)
    

Retrieved Document:
Title: Eh Janam Tumhare Lekhe
        Type: Movie
        Director: Harjit Singh
        Cast: Pavan Malhotra, Sudhanshu Aggarwal, Arjuna Bhalla, Avrinder Kaur, Arvinder Bhatti, Master Yuvraj, Jai Bharti, Gagandeep Singh
        Country: India
        Release Year: 2015
        Rating: TV-14
        Duration: 124 min
        Genres: Dramas, International Movies
        Description: Driven by the lessons he learned from his mother, and the values of the Sikh religion, a man sets out on a mission to serve humanity.
Metadata: {'show_id': 's6668', 'type': 'Movie', 'country': 'India', 'release_year': 2015, 'rating': 'TV-14', 'listed_in': 'Dramas, International Movies'}
--------------------------------------------------
Retrieved Document:
Title: Chauthi Koot
        Type: Movie
        Director: Gurvinder Singh
        Cast: Suvinder Vikky, Rajbir Kaur, Gurpreet Kaur Bhangu, Taranjit Singh, Harleen Kaur, Kanwaljeet Singh, Harnek Aulakh, Tejpal Singh, Gulshan Saggi
      

In [18]:
context = "\n\n".join([doc.page_content for doc in retrieved_docs])
prompt = f"""
You are a helpful assistant. Answer the question using the provided information.

Context:
{context}

Question: {query}
Answer:
"""

In [19]:
response = llm(prompt, max_tokens=256) 
print("AI Response:", response["choices"][0]["text"])

Llama.generate: 1 prefix-match hit, remaining 824 prompt tokens to eval
llama_perf_context_print:        load time =    1688.75 ms
llama_perf_context_print: prompt eval time =   77820.90 ms /   824 tokens (   94.44 ms per token,    10.59 tokens per second)
llama_perf_context_print:        eval time =   16617.60 ms /    78 runs   (  213.05 ms per token,     4.69 tokens per second)
llama_perf_context_print:       total time =   94469.99 ms /   902 tokens


AI Response: 
It is not clear which of the provided movie titles is the latest blockbuster movie made by a Sikh person. The latest release year among the provided movies is 2015 for "Eh Janam Tumhare Lekhe" and "Chauthi Koot," and 2014 for "Chaar Sahibzaade."


In [20]:
# step_back_prompt = PromptTemplate(
#     input_variables=["original_question"],
#     template=(
#         "You are an expert at reformulating questions.\n"
#         "User's question: \"{original_question}\"\n"
#         "Step back and provide a more general question that captures the essence of the user's query."
#     )
# )
# step_back_chain = LLMChain(llm=llm, prompt=step_back_prompt)

In [21]:
# from langchain.llms import OpenAI
# # decouple to read .env variables(OpenAI Key)
# from decouple import config
# # import openAI from langChain
# from langchain.llms import OpenAI
# # import prompt template
# from langchain import PromptTemplate

# # create the prompt
# prompt_template: str = """/
# You are a vehicle mechanic, give responses to the following/ 
# question: {question}. Do not use technical words, give easy/
# to understand responses.
# """

# prompt = PromptTemplate.from_template(template=prompt_template)

# # format the prompt to add variable values
# prompt_formatted_str: str = prompt.format(
#     question="Why won't a vehicle start on ignition?")

# # instantiate the OpenAI intance
# llm = OpenAI(openai_api_key=config("OPANAI_API_KEY"))

# # make a prediction
# prediction = llm.predict(prompt_formatted_str)

# # print the prediction
# print(prediction)

In [22]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.llms import LlamaCpp

# Load the model using LangChain's LlamaCpp wrapper
llm = LlamaCpp(
    model_path=model_path,  # Ensure the model path is correct
    n_ctx=2048,
    n_threads=6
)

# Define the Step-Back Prompt
step_back_prompt = PromptTemplate(
    input_variables=["original_question"],
    template=(
        "You are an expert at reformulating questions.\n"
        "User's question: \"{original_question}\"\n"
        "Step back and provide a more general question that captures the essence of the user's query."
    )
)

# Create the LLM Chain
step_back_chain = LLMChain(llm=llm, prompt=step_back_prompt)

# Example Usage
# query = "How has the use of saffron in desserts changed over the last year, according to restaurant menus or news articles?"
step_back_response = step_back_chain.run(original_question=query)

print("Step-Back Reformulated Question:", step_back_response)


llama_model_load_from_file_impl: using device Metal (AMD Radeon Pro 5300M) - 3143 MiB free
llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from /Users/gauravbindra/.cache/huggingface/hub/models--TheBloke--Mistral-7B-Instruct-v0.1-GGUF/snapshots/731a9fc8f06f5f5e2db8a0cf9d256197eb6e05d1/mistral-7b-instruct-v0.1.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.1
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loade

Step-Back Reformulated Question: 
"Who are some of the most successful filmmakers from the Sikh community and what are their recent projects?"


In [27]:
from langchain.prompts import FewShotPromptTemplate, PromptTemplate
from langchain.chains import LLMChain
from langchain.llms import LlamaCpp

# ✅ Load the model using LangChain's LlamaCpp wrapper
model_path = "/Users/gauravbindra/.cache/huggingface/hub/models--TheBloke--Mistral-7B-Instruct-v0.1-GGUF/snapshots/731a9fc8f06f5f5e2db8a0cf9d256197eb6e05d1/mistral-7b-instruct-v0.1.Q4_K_M.gguf"

llm = LlamaCpp(
    model_path=model_path,  # Ensure the model path is correct
    n_ctx=2048,
    n_threads=6
)

# ✅ Define Few-Shot Examples for Step-Back Prompting
examples = [
    {
        "original_query": "What are the chemical properties of the element discovered by Marie Curie?",
        "step_back_query": "What elements were discovered by Marie Curie, and what are their chemical properties?"
    },
    {
        "original_query": "Why does my LangGraph agent `astream_events` return a long trace instead of the expected output?",
        "step_back_query": "How does the `astream_events` function work in LangGraph agents?"
    },
    {
        "original_query": "Which school did Estella Leopold attend from August to November 1954?",
        "step_back_query": "What is the educational history of Estella Leopold?"
    }
]

# ✅ Create a Few-Shot Prompt Template
few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=PromptTemplate(
        input_variables=["original_query", "step_back_query"],
        template="User Query: {original_query}\nStep-Back Query: {step_back_query}\n"
    ),
    prefix="You are an expert at reformulating questions into more general queries to improve retrieval.\nHere are some examples:",
    suffix="Now, step back and provide a more general question that captures the essence of the user's query. Just give the query. \n User Query: {original_query}\nStep-Back Query:",
    input_variables=["original_query"]
)

# ✅ Create LLMChain for Step-Back Prompting
step_back_chain = LLMChain(llm=llm, prompt=few_shot_prompt)

# ✅ Example Usage
# query = "How has the use of saffron in desserts changed over the last year, according to restaurant menus or news articles?"
step_back_response2 = step_back_chain.run(original_query=query)

print("\n🔹 Step-Back Reformulated Question:", step_back_response2.strip())


llama_model_load_from_file_impl: using device Metal (AMD Radeon Pro 5300M) - 3141 MiB free
llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from /Users/gauravbindra/.cache/huggingface/hub/models--TheBloke--Mistral-7B-Instruct-v0.1-GGUF/snapshots/731a9fc8f06f5f5e2db8a0cf9d256197eb6e05d1/mistral-7b-instruct-v0.1.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.1
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loade


🔹 Step-Back Reformulated Question: Who are Sikh filmmakers, and what are their recent blockbuster movies?


In [23]:
retriever = vector_store.as_retriever(search_kwargs={"k": 5})  # Retrieve top-5 relevant docs


In [28]:
def hybrid_retrieval(user_query: str):
    """Retrieves documents using both original and step-back queries."""
    # Step 1: Generate step-back query
    # step_back_response = step_back_chain.run(original_question=query)
    step_back_query = step_back_chain.run(original_question=query)
    print(f"Step-back query: {step_back_query.strip()}")  # Debugging

    # Step 2: Retrieve documents using both queries
    docs_original = retriever.get_relevant_documents(query)
    docs_step_back = retriever.get_relevant_documents(step_back_query)

    # Step 3: Merge and deduplicate documents
    unique_docs = {doc.page_content: doc for doc in (docs_original + docs_step_back)}
    combined_docs = list(unique_docs.values())

    return combined_docs  # These will be passed to the LLM


In [29]:
retrieved_docs = hybrid_retrieval(query)

print("Final Retrieved Documents:")
for doc in retrieved_docs:
    print(doc.page_content[:200] + "...")  # Display snippet

ValueError: Missing some input keys: {'original_query'}